In [1]:
# used findpark because python did not know where our spark installation was. 
import findspark
findspark.init()

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 
spark = (SparkSession.builder.appName("shopping").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-03-24 08:14:52,698 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#schema = ["business_id STRING ,name STRING, address STRING, city STRING,"state":"CA","postal_code":"93101","latitude":34.4266787,"longitude":-119.7111968,"stars":5.0,"review_count":7,"is_open":0,"attributes":{"ByAppointmentOnly":"True"},"categories":"Doctors, Traditional Chinese Medicine, Naturopathic\/Holistic, Acupuncture, Health & Medical, Nutritionists","hours":null}"]
from pyspark.sql.types import *
review_schema = StructType([StructField("review_id", StringType(), False),
      StructField("user_id", StringType(), False),
      StructField("business_id", StringType(), False),
      StructField("stars", StringType(), False), 
      StructField("useful", IntegerType(), False),
      StructField("funny", IntegerType(), False),
      StructField("cool", IntegerType(), False),
      StructField("text", StringType(), False),
      StructField("date", StringType(), False),])
review_df = spark.read.csv("hdfs://namenode:9000/project_data/review.csv", sep = '|', header = False, schema = review_schema)
review_df.show(5)

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|MmmrqbdupYBJq9jnl...|TjD2S66AWxlKsPSQB...|znK6tMeegKf9YnGut...|  5.0|     0|    0|   0|Great little spot...|2021-04-16 22:02:21|
|1_uWX0bBJNnZjPEcp...|Vtfxxq3nxdYHRiH6O...|UxwpCVLgPWCeaRyet...|  5.0|     0|    0|   0|James was super k...|2021-01-03 21:54:48|
|WVAEE47MnM3Sv2cvM...|ZDw_qN5Fy6PE0gdbV...|eVI64EQymywsvMLmD...|  5.0|     0|    0|   0|By far the best i...|2020-11-01 19:50:03|
|D5b6iooZZcJ8nG-9P...|UvMDlX2wV4Md9OwGW...|SPcPJfPgWzhjUDqVF...|  5.0|     0|    0|   0|I thought that th...|2021-03-24 05:53:49|
|uDxnT7zWFnxEDxp5e...|ir9ixBZPzBwWe9IZ_...|T5XzQ6YnVExvd0BOR...|  5.0|     0|    0|   0|As

In [10]:
# DONT RUN THIS AGAIN
review_df.write.mode("overwrite").saveAsTable('reviews')

In [7]:
broken = (review_df
          .where(F.col("business_id").isNull())
)
# broken = (df
#           .where(F.col("review_id").contains('One small area of')))

In [8]:
broken.show(5)

+--------------------+-------+-----------+-----+------+-----+----+----+----+
|           review_id|user_id|business_id|stars|useful|funny|cool|text|date|
+--------------------+-------+-----------+-----+------+-----+----+----+----+
|ys8wYr2V++++failed\t|   null|       null| null|  null| null|null|null|null|
|jfVm-Qep++++failed\t|   null|       null| null|  null| null|null|null|null|
|m74VYMFn++++failed\t|   null|       null| null|  null| null|null|null|null|
+--------------------+-------+-----------+-----+------+-----+----+----+----+



In [33]:
parquet_table = "review_table_parquet" # name of the table 
review_df.write.format("parquet").saveAsTable(parquet_table)

In [34]:
spark.read.table("review_table_parquet").take(1)

[Row(review_id='6fObpwIggOQR1oDapyQzyg', user_id='lN-1uUHeV_QyFbczw8ErlA', business_id='grpNey31cTGKrhmQQaNGdA', stars='5.0', useful=0, funny=0, cool=0, text='Had a wonderful, authentic basque dinner Friday night.  Service was great and friendly and the bar was fun. Thanks Gavin!  We will definitely be back!Chris and Cathy (Turlock CA)', date='2020-06-28 19:45:32')]

In [35]:
spark.sql("DROP TABLE IF EXISTS review_table_parquet")

DataFrame[]

In [11]:
broken.count()

3

In [3]:

business_schema = StructType([
    StructField("business_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("hours", StringType(), True)
])
business_df = spark.read.csv("hdfs://namenode:9000/project_data/business.csv", sep = '|', header = False, schema = business_schema)
business_df.show(5)

+--------------------+--------------------+--------------------+------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|         business_id|                name|             address|        city|state|postal_code|     latitude|     longitude|stars|review_count|is_open|          attributes|          categories|               hours|
+--------------------+--------------------+--------------------+------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|O4dxdtqo625JFN0uF...|Kelly's Bake Shop...|        1582 Main St|     Dunedin|   FL|      34698|    28.019895|   -82.7610784|  4.5|          49|      0|"{'BusinessParkin...|Sandwiches, Resta...|{'Tuesday': '7:0-...|
|Q0k9AkPe9PWfGNvhd...|            3D Vapor|   105 E Vandalia St|Edwardsville|   IL|      62025|   38.8115422|    -89.955539|  3.5|          

In [ ]:
# DONT RUN THIS AGAIN
business_df.write.mode("overwrite").saveAsTable('business')

In [25]:
broken_business = (business_df
          .where(F.col("business_id").isNull() | F.col("categories").isNull())
)

In [26]:
broken_business.show(5)

+-----------+----+-------+----+-----+-----------+--------+---------+-----+------------+-------+----------+----------+-----+
|business_id|name|address|city|state|postal_code|latitude|longitude|stars|review_count|is_open|attributes|categories|hours|
+-----------+----+-------+----+-----+-----------+--------+---------+-----+------------+-------+----------+----------+-----+
+-----------+----+-------+----+-----+-----------+--------+---------+-----+------------+-------+----------+----------+-----+



In [11]:
# combining two tables
combined_table = spark.sql("SELECT text FROM reviews")

In [3]:
business=spark.read.parquet('hdfs://namenode:9000/project_data/data/business/')
business.printSchema()

# TODO how to read parquet in sql
# combined_table = spark.sql("SELECT name FROM business")

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- attributes: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- hours: string (nullable = true)



In [4]:
type(business)

pyspark.sql.dataframe.DataFrame

In [5]:
# create temporary table to use sql 
business.createOrReplaceTempView("business")
print(type(business))
btest = spark.sql("SELECT name FROM business")

<class 'pyspark.sql.dataframe.DataFrame'>


In [30]:
business.count()

150346

In [24]:
reviews =spark.read.parquet('hdfs://namenode:9000/project_data/data/reviews/')

In [31]:
reviews.count()

6990280

In [32]:
reviews.createOrReplaceTempView("reviews")

# TODO what is suitable ON or WHERE??? (ON is the suitable operation)
combined_table = spark.sql("SELECT r.text, b.business_id, b.categories FROM reviews AS r LEFT JOIN business AS b ON b.business_id = r.business_id ")

In [33]:
combined_table.count()

6990280

In [28]:
combined_table_1 = combined_table.where(F.col('categories').contains("Restaurants"))

In [29]:
combined_table_1.count()

4724468

In [1]:
import findspark
findspark.init()

import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

ModuleNotFoundError: No module named 'delta'

In [6]:
from pyspark.sql.types import *
business_schema = StructType([
    StructField("business_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("hours", StringType(), True)
])
business_df = spark.read.csv("hdfs://namenode:9000/project_data/business.csv", sep = '|', header = False, schema = business_schema)
business_df.show(5)

+--------------------+--------------------+--------------------+------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|         business_id|                name|             address|        city|state|postal_code|     latitude|     longitude|stars|review_count|is_open|          attributes|          categories|               hours|
+--------------------+--------------------+--------------------+------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+
|O4dxdtqo625JFN0uF...|Kelly's Bake Shop...|        1582 Main St|     Dunedin|   FL|      34698|    28.019895|   -82.7610784|  4.5|          49|      0|"{'BusinessParkin...|Sandwiches, Resta...|{'Tuesday': '7:0-...|
|Q0k9AkPe9PWfGNvhd...|            3D Vapor|   105 E Vandalia St|Edwardsville|   IL|      62025|   38.8115422|    -89.955539|  3.5|          

In [1]:
import findspark
findspark.init("/usr/local/spark")

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 

from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .master("spark://namenode:7077")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9c0b034e-6912-49f8-b913-d902a90955db;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 290ms :: artifacts dl 17ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|ev

In [2]:
business=spark.read.format("parquet").load('hdfs://namenode:9000/project_data/data/business/')

Py4JJavaError: An error occurred while calling o45.load.
: java.lang.NoSuchMethodError: org.apache.spark.sql.internal.SQLConf$.PARQUET_FIELD_ID_READ_ENABLED()Lorg/apache/spark/internal/config/ConfigEntry;
	at io.delta.sql.DeltaSparkSessionExtension.$anonfun$apply$3(DeltaSparkSessionExtension.scala:88)
	at org.apache.spark.sql.SparkSessionExtensions.$anonfun$buildResolutionRules$1(SparkSessionExtensions.scala:174)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.sql.SparkSessionExtensions.buildResolutionRules(Spar